<a href="https://colab.research.google.com/github/yebyyy/English-Premier-League-2023-2024-Prediction/blob/main/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This code is for scraping the data for the current season of English Premier League

In [44]:
import requests

In [45]:
import pandas as pd

In [46]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [47]:
data = requests.get(standings_url)

In [48]:
from bs4 import BeautifulSoup

In [49]:
soup = BeautifulSoup(data.text)

#### Now we need give the soup object something to select.
Since everything is inside the table html object, we need to select it and then get everything anchor tags for the clubs.

In [50]:
# selecting the table using css selector
standings_table = soup.select("table.stats_table")[0] # .stats_table is the class name

In [51]:
# find all the <a>(the clubs) in the standings table
links = standings_table.find_all("a") # find_all only finds tags

In [52]:
# links should actually contain all the links
links = [l.get("href") for l in links]

In [53]:
# links should actually contain the links that have clubs
links = [l for l in links if "/squads/" in l]
links

['/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/e297cd13/Luton-Town-Stats',
 '/en/squads/943e8050/Burnley-Stats',
 '/en/squads/1df6b87e/Sheffield-United-Stats']

In [54]:
# since the links only have the subdomain, we should include the first half of the url as well
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/s

#### Using Pandas and Requests to extract match stats: Using a single team as an example

In [55]:
team_url_first = team_urls[0]

In [56]:
first_team_data = requests.get(team_url_first)

Now we use pandas for the parsing and read only the table called scores and fixtures since we only care about the data there

In [57]:
matches_first_team = pd.read_html(first_team_data.text, match="Scores & Fixtures")

Since this is a list, we want a pandas dataFrame, therefore we get the first element from this list

In [58]:
matches_first_team[0]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,1.3,1.4,35.0,40096.0,Virgil van Dijk,4-3-3,Anthony Taylor,Match Report,NaN
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,3.0,1.3,64.0,53145.0,Virgil van Dijk,4-3-3,Thomas Bramall,Match Report,NaN
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,0.9,2.0,41.0,52214.0,Virgil van Dijk,4-3-3,John Brooks,Match Report,NaN
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,2.5,0.7,63.0,50109.0,Trent Alexander-Arnold,4-3-3,Simon Hooper,Match Report,NaN
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,2.5,0.6,65.0,31257.0,Andrew Robertson,4-3-3,Michael Oliver,Match Report,NaN
5,2023-09-21,18:45,Europa Lg,Group stage,Thu,Away,W,3.0,1.0,at LASK,2.2,0.3,66.0,18091.0,Virgil van Dijk,4-3-3,Marco Di Bello,Match Report,NaN
6,2023-09-24,14:00,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,West Ham,3.0,1.1,63.0,50136.0,Virgil van Dijk,4-3-3,Chris Kavanagh,Match Report,NaN
7,2023-09-27,19:45,EFL Cup,Third round,Wed,Home,W,3.0,1.0,Leicester City,NaN,NaN,57.0,49732.0,Curtis Jones,4-3-3,Tim Robinson,Match Report,NaN
8,2023-09-30,17:30,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Tottenham,1.3,2.2,36.0,62001.0,Virgil van Dijk,4-3-3,Simon Hooper,Match Report,NaN
9,2023-10-05,20:00,Europa Lg,Group stage,Thu,Home,W,2.0,0.0,be Union SG,2.9,1.1,72.0,49513.0,Trent Alexander-Arnold,4-3-3,Morten Krogh,Match Report,NaN


#### Now we also care about the shooting data, so we continue using the same team for the example

Since the shooting page is now just a URL in an anchor in the webpage of the club page we were working on, we find all the links and keep the one that has shooting in it

In [59]:
soup_for_first_team = BeautifulSoup(first_team_data.text)